In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from EngbertMicrosaccadeToolbox import microsac_detection

In [2]:
# Set parameters
SAMPLING = 500
MINDUR = 3
VFAC = 5

In [3]:
# Read raw data (ASCII; subject 01, trial 005)
d = pd.read_csv("data/f01.005.dat", sep="\t", header=None, names=["t", "x_l", "y_l", "x_r", "y_r"])
d.head()
d = d.loc[3000:4499]
d.head()

,t,x_l,y_l,x_r,y_r
3000,6000.0,0.01188,0.60588,-0.21384,0.41976
3001,6002.0,0.02376,0.61380,-0.20988,0.41580
3002,6004.0,0.02772,0.62172,-0.19800,0.41976
3003,6006.0,0.03564,0.71280,-0.20196,0.41184
3004,6008.0,0.06336,0.64152,-0.20196,0.42372


In [4]:
np.asarray(d.x_r).shape

(1500,)

In [5]:
aa = np.vstack([np.asarray(d.x_r), np.asarray(d.y_r)]).T

In [9]:
np.save( "xr.npy", aa,)

In [11]:
np.genfromtxt("/Users/lisa/Documents/SFB1294_B05/EngbertMSDET_python/EngbertMicrosaccadeToolbox/tests/xrs_smooth.dat")

array([[-0.21384,  0.41976],
       [-0.20592,  0.41976],
       [-0.20262,  0.41976],
       ...,
       [-0.07722, -0.03762],
       [-0.0792 , -0.02574],
       [-0.0792 , -0.02574]])

In [6]:
bb = microsac_detection.smoothdata(aa)

(1500, 2)


In [7]:
bb

array([[-0.21384,  0.41976],
       [-0.20592,  0.41976],
       [-0.20262,  0.41976],
       ...,
       [-0.07722, -0.03762],
       [-0.0792 , -0.02574],
       [-0.0792 , -0.02574]])

In [13]:
w = np.array([[1,2,3], [1,2,3]]).T
w.cumsum(axis=0)

array([[1, 1],
       [3, 3],
       [6, 6]])

In [68]:
aa[0:10]

array([[-0.21384,  0.41976],
       [-0.20988,  0.4158 ],
       [-0.198  ,  0.41976],
       [-0.20196,  0.41184],
       [-0.20196,  0.42372],
       [-0.20196,  0.42372],
       [-0.20196,  0.42372],
       [-0.19008,  0.42372],
       [-0.18612,  0.41976],
       [-0.15444,  0.4158 ]])

In [109]:
def vecvel(x, sampling):
  N, _ = x.shape
  v = np.zeros((N, 2))
  print(v.shape)
  v[2:(N-2),] = SAMPLING/6*(x[4:N,] + x[3:(N-1),] - x[1:(N-3),] - x[0:(N-4),])
  v[1,] = SAMPLING/2*(x[2,] - x[0,])
  v[(N-2),] = SAMPLING/2*(x[N-1,] - x[(N-3),])  
  return v
  

In [110]:
N, _ = aa.shape
aa[4:N-1,].shape
aa[3:N-2,].shape
aa[1:(N-4),].shape
aa[0:(N-5),].shape

(1495, 2)

In [111]:
aa[N-1,1]

-0.02376

In [112]:
aa[4:N-1,]

array([[-0.20196,  0.42372],
       [-0.20196,  0.42372],
       [-0.20196,  0.42372],
       ...,
       [-0.06336, -0.04356],
       [-0.07524, -0.04752],
       [-0.08316, -0.0396 ]])

In [113]:
aa = np.array([[1,2,3,4, 5, 6],[1,2,3,4,5, 6]]).T
aa.shape

(6, 2)